### Download Sentinel-2 image collection

In [ ]:
import geemap
import json
import os
import requests
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
import pandas as pd


In [ ]:
#initialize GEE using your Google Account
ee.Authenticate()
ee.Initialize()

In [ ]:
Map = geemap.Map(center=[-27.93186,32.48897],zoom=9)

file_path = './data/boundaries/isimangaliso_bbox.geojson'
with open(file_path) as f:
    bbox = geojson_to_ee(json.load(f))
    
Map.addLayer(bbox, {}, "Isimangaliso Wetland Park")
Map

In [ ]:
collection = ee.ImageCollection("COPERNICUS/S2_SR") \
    .filterBounds(bbox).filterDate('2019-01-01','2019-01-06')

In [ ]:
collection_list = collection.toList(collection.size())
latest_meta = collection_list.getInfo()[-1]

In [ ]:
def get_meta_s2(meta):
    product_id = meta['properties']['PRODUCT_ID']
    date = pd.to_datetime(product_id.split('_')[2].split('T')[0]).date()
    tile = product_id.split('_')[5]
    cloud_cover = meta['properties']['CLOUD_COVERAGE_ASSESSMENT']
    data = {'id':product_id,'date':date,'tile':tile,'cloudcover':cloud_cover}
    df = pd.DataFrame(data,index=[0])
    return df

In [ ]:
products = list(map(get_meta_s2,collection_list.getInfo()))
products_df = pd.concat(products).reset_index(drop=True)

In [ ]:
products_df